In [24]:
from pathlib import Path
from xpinn import XPINN
import optax
from type_util import Array
from jax import hessian, jit, vmap, grad
import jax.numpy as np
import numpy as onp
from typing import Callable, Tuple
from type_util import Array, Params
from utils import data_path
import json

from jax import config

config.update("jax_enable_x64", True)

In [4]:
from utils import data_path

file = data_path / "navier_stokes_pinn_train.json"
file_test = data_path / "navier_stokes_test.json"
activation = np.tanh
xpinn = XPINN(file, activation)

In [5]:
from typing import Callable
from type_util import Params
from base_network import neural_network

LFunc = Callable[[Params, dict[str, Array]], Array]

model = neural_network(activation)
v_model = vmap(model, (None, 0))


We require a lot of different expresions for calculating the rather ugly differential conditions imposed by the navier stokes. Below is a sketch for how they may be calculated. 


In [7]:
def psi_func(params, xyt):
    return v_model(params, xyt)[0]

hess_psi = hessian(psi_func, argnums=1)
d_psi_dxyt = grad(psi_func, argnums=1)

u_x = lambda params, xyt: hess_psi(params, xyt)[0,1] #psi_yx
u_y = lambda params, xyt: hess_psi(params, xyt)[2,2] #psi_yy
u_xx = lambda params, xyt: grad(u_x, argnums=1)(params, xyt)[0] #psi_yxx
u_t = lambda params, xyt: hess_psi(params, xyt)[1,3] #psi_ty
u_yy = lambda params, xyt: grad(u_y, argnums=1)(params, xyt)[1] #psi_yyy

v_y = lambda params, xyt: -hess_psi(params, xyt)[0,1] #-psi_yx
v_x = lambda params, xyt: -hess_psi(params, xyt)[0,0] #-psi_xx
v_t = lambda params, xyt: -hess_psi(params, xyt)[1,3] #-psi_ty
v_xx = lambda params, xyt: grad(v_x, argnums=1)(params, xyt)[0] #-psi_xxx
v_yy = lambda params, xyt: grad(v_y, argnums=1)(params, xyt)[1] #-psi_yyy


u = lambda params, xyt: d_psi_dxyt(params, xyt)[1]
v = lambda params, xyt: -d_psi_dxyt(params, xyt)[0]

And here are the functions relating to the preassure p

In [8]:
p = lambda params, xyt: v_model(params, xyt)[1]

p_x = lambda params, xyt: grad(p, argnums=1)(params, xyt)[0]
p_y = lambda params, xyt: grad(p, argnums=1)(params, xyt)[1]

Two different residuals ?  The first is the one from PINNS(II) paper given by 
$$
    f = u_t + u  u_x + v  u_y + p_x - \nu (u_{xx} + u_{yy}) = 0
$$

$$
g = v_t + u  v_x + v  v_y + p_y - \nu (v_{xx} + v_{yy}) = 0
$$


Dortmunt PDE tutorial loss (@junmiaoHu verify please idk)

$$
\frac{\partial \mathbf{u}}{\partial t} - \nu \Delta \mathbf{u} + (\mathbf{u} \cdot \nabla) \mathbf{u} + \nabla p = \boldsymbol{0}
$$
Here $\mathbf{u}$ is a vector containing both the x and y component of the flow, which I've previously denoted as u and v. So using our notation the expression would be

$$
f = u_t - \nu (u_{xx} + u_{yy}) + u u_x + v u_y + p_x = 0 
$$ 
$$ g = v_t - \nu (v_{xx} + v_{yy}) + u v_x + v v_y + p_y = 0 
$$


So the two resiudals are the same! jippi I'm not crazy


In [9]:
def navier_stokes_residual_factory(nu: float) -> Tuple[Callable[[Params, dict[str, Array]], Array],
                                                       Callable[[Params, dict[str, Array]], Array]]:
    def f(params, xyt):
        return (u_t(params, xyt) + u(params, xyt)*u_x(params, xyt) + 
                v(params, xyt)*u_y(params, xyt) + p_x(params, xyt) - 
                nu*(u_xx(params, xyt) + u_yy(params, xyt)))
    def g(params, xyt):
        return (v_t(params, xyt) + u(params, xyt)*v_x(params, xyt) + 
                v(params, xyt)*v_y(params, xyt) + p_y(params, xyt) - 
                nu*(v_xx(params, xyt) + v_yy(params, xyt)))
    return f, g

Now for the boundary loss. Idea: we have sorted the array and know the indecies for which the different boundaries start and end. Thus we can pass these indecies as arguments in the function and treat the different parts of the array differently

Let's precompute the inflow

In [37]:
U_y = 1 #idk what this constant is suppsed to be @junmiaoHelp

inflow_func = lambda y: 4 * U_y * y * (0.41 - y)/0.41**2
v_inflow_func = vmap(inflow_func)

def calculate_inflow(v_inflow_func, file):
        inflow = []
        with open(file, "r") as infile:
                data = json.load(infile)
                for i in range(len(data['XPINNs'])):
                        left_pts = data['XPINNs'][i]['left boundary']
                        inflow.append(v_inflow_func(np.array(left_pts)[:,1]))
        return inflow

file = data_path / "navier_stokes_pinn_train.json"
inflow = calculate_inflow(v_inflow_func, file)


In [ ]:
def boundary_loss_factory(inflow: Callable[[Array], Array]) -> Tuple[Callable[[Params, dict[str, Array]], Array],
                                                                    Callable[[Params, dict[str, Array]], Array]]:
    def left_boundary_loss(params, xyt):
        return (u(params, xyt) - inflow_func(xyt[1]))**2
    
    def top_and_bottom_boundary_loss(params, xyt):
        return u(params, xyt)**2
    
    def cylinder_boundary_loss(params, xyt):
        
        
    def right_boundary_loss(params, xyt):
        return 
    
    return 
    

In [22]:
def boundary_loss(params: Params, points: dict[str, Array]) -> Array:
    return (left_boundary_loss(params, points['left boundary']) + 
            right_boundary_loss(params, points['right boundary']) +
            top_and_bottom_boundary_loss(params, points['top and bottom boundary']) +
            cylinder_boundary_loss(params, points['cylinder boundary']))
    

In [ ]:
def boundary_loss_factory(target: float | Array) -> LFunc:
    
    def boundary_loss(params: Params, points: dict[str, Array]) -> Array:
        pts = points["boundary"]
        eval = v_model(params, pts)
        return np.mean((eval - target) ** 2)

    return boundary_loss

Below is a shitty atempt of converting tensorflow code from PINNS(II) paper. Unusable trash

In [ ]:
def navier_stokes_functional(model, nu):
    """
    model: The neural network model that takes (x, y, t) inputs.
    nu: The kinematic viscosity.
    
    returns relevant values for loss and simulation.
    """
    # Define functions for u and v derivatives
    def psi_component(model, params, xyt):
        psi = model(params, xyt)[:, 0]  # Assuming first component is psi
        return psi
    
    # Compute u and v from the stream function psi
    d_psi = grad(psi_component, argnums=(2,))
    
    def u(model, params, xyt):
        return -d_psi(model, params, xyt)[1]
    
    def v(model, params, xyt):
        return d_psi(model, params, xyt)[0]
    
    # Create functions for pressure gradient components
    def p_component(model, params, xyt):
        p = model(params, xyt)[:, 1]  # Assuming second component is pressure
        return p
    
    d_p = grad(p_component, argnums=(2,))
    
    # Higher order derivatives for u and v
    def u_t(model, params, xyt):
        return grad(u, argnums=(2,))(model, params, xyt)
    
    def u_x(model, params, xyt):
        return grad(u, argnums=(2,))(model, params, xyt)[0]
    
    def u_y(model, params, xyt):
        return grad(u, argnums=(2,))(model, params, xyt)[1]
    
    def u_xx(model, params, xyt):
        return grad(u_x, argnums=(2,))(model, params, xyt)[0]

    def u_yy(model, params, xyt):
        return grad(u_y, argnums=(2,))(model, params, xyt)[1]

    def v_t(model, params, xyt):
        return grad(v, argnums=(2,))(model, params, xyt)
    
    def v_x(model, params, xyt):
        return grad(v, argnums=(2,))(model, params, xyt)[0]

    def v_y(model, params, xyt):
        return grad(v, argnums=(2,))(model, params, xyt)[1]
    
    def v_xx(model, params, xyt):
        return grad(v_x, argnums=(2,))(model, params, xyt)[0]
    
    def v_yy(model, params, xyt):
        return grad(v_y, argnums=(2,))(model, params, xyt)[1]

    # Assemble the Navier-Stokes functional
    def function(params, xyt):
        # Calculate velocity components using the stream function
        u_vel = u(model, params, xyt)
        v_vel = v(model, params, xyt)

        # Calculate pressure gradient components
        p_x = d_p(model, params, xyt)[0]
        p_y = d_p(model, params, xyt)[1]

        # Calculate time and space derivatives of velocity components
        u_t_val = u_t(model, params, xyt)
        u_x_val = u_x(model, params, xyt)
        u_y_val = u_y(model, params, xyt)
        u_xx_val = u_xx(model, params, xyt)
        u_yy_val = u_yy(model, params, xyt)
        
        v_t_val = v_t(model, params, xyt)
        v_x_val = v_x(model, params, xyt)
        v_y_val = v_y(model, params, xyt)
        v_xx_val = v_xx(model, params, xyt)
        v_yy_val = v_yy(model, params, xyt)

        # Compute the residuum of the Navier-Stokes equations
        f = u_t_val + u_vel * u_x_val + v_vel * u_y_val + p_x - nu * (u_xx_val + u_yy_val)
        g = v_t_val + u_vel * v_x_val + v_vel * v_y_val + p_y - nu * (v_xx_val + v_yy_val)

        return u_vel, v_vel, p_x, p_y, f, g

    return jit(function)
